Design for Co-Factor Swapping.
Simulating co-factor swapping. The objective is to investigate if productivity can be improved by swapping NADPH for NADH as a cofactor in the reactions requiring CYP71AV1. The co-factor swap analysis will determine if an increase in the available NADPH (caused by altering the cofactor specificity of CYP71AV1 to favor NADH over NADPH) will lead to increased production of dihydroartemisinic acid, our target compound. Co-factor swapping 

#TEXT FROM EXAMPLE: The conversion of tryptamine to 4-hydroxy-tryptamine uses NADPH as a cofactor, and therefore a cofactor swap analysis was performed in order to determine whether increasing the pool of available NADPH by swapping the cofactor specificty of reactions in the model from NADPH to NADH could increase psilocybin production. This hypothesis is supported by research by King et al., which showed that swapping the cofactor specificity from NAD(H) to NADP(H) in oxidoreductase enzymes involved in central carbon metabolism can improve the theoretical yields of several amino acids including L-aspartate, L-lysine, and L-isoleucine as well as some non-native products in S. cerevisiae (King, Feist, 2014). CofactorSwapOptimization utilizes an algorithm that identifies which reactions in the model can undergo a swap in cofactor specificity in order to increase the availability of a cofactor that is rate-limiting for the production of a target metabolite, and increase its theoretical yield.

In [1]:
%run 1_incorporation_of_heterologous_genes.ipynb
#initial run takes some time to process
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

,equation,lower_bound,upper_bound
MNXR69185,geranyl diphosphate + isopentenyl diphosphate ...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR55238,geranyl diphosphate + diphosphate <=> dimethyl...,-1000.0,1000.0


grdp_c + ipdp_c --> frdp_c + ppi_c
frdp_c --> amorph_c + ppi_c
amorph_c + nadp_c + o2_c --> artalc_c + h2o_c + nadph_c
artalc_c + nadp_c + o2_c --> artald_c + 2 h2o_c + nadph_c
artald_c + h_c + nadph_c --> dihydroartald_c + nadp_c
artalc_c + nadp_c + o2_c --> dihydroartaci_c + h2o_c + nadph_c
artalc_c + nadp_c + o2_c --> artaci_c + h2o_c + nadph_c
0.36342857142857143


In [2]:
model = read_sbml_model("data/iYO844_modified.xml")

In [3]:
model.solver = "glpk" #specifying the solver for optimizing to be used as GLPK (GNU Linear Programming Kit)
biomass = model.reactions.BIOMASS_BS_10 #specifying our objective: optimizing biomass formation
biomass.lower_bound = 0.1 
demand = model.reactions.DM_dihydroartaci_c #i tvivl om, hvad der skal stå her, før stod der demand = model.reactions.DM_psi
#We need to define the demand reaction for the "target" (Dihydroartemisinic acid) in "01-incorporation.....ipynb"
model.objective = demand
prod_yield = product_yield(demand, model.reactions.EX_glc__D_e) #Her skal vi have vores nye boundary, som optimerer for produktionen af vores ønskede stof

In [4]:
#CofactorSwapOptimization? This command tells us what the command does

In [5]:
# Swapping NADPH to NADH                                                                               
swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True, cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']))
#we may not need to define the cofactor_id_swaps, as the ids in our model are the same as the default (but will it know what to change???)

In [6]:
%%time
result = swap_opt.run(max_size=5) #can be changed, but I'M NOT QUITE SURE WHAT EXACTLY CHANGES (looks like time in min)
#we get no result right now, meaning nothing was found.... Can that be right?

Starting optimization at Wed, 15 Nov 2023 22:08:21


HBox()

Finished after 00:00:58
Wall time: 1min 42s


In [16]:
#print(f'The results of co-factor swapping are: 'result)
result

,index,targets,fitness


In [8]:
#testing the reactions appearing in the result by using "model.reactions.r252" 

Old attempt, kept just in case

In [9]:
#Uploading cameo package
#from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

Model setup

In [10]:
#from cobra.io import read_sbml_model, write_sbml_model #importing functions needed
#from cobra import Reaction, Metabolite

In [11]:
# Loading the heterologous GEM built from the original model
#model = read_sbml_model('data/iYO844_modified.xml')

In [12]:
# Defining the reaction producing our final product as the objective.
#objective_reaction = model.reactions.ALDH1_CYP71AV1
#model.objective = objective_reaction
#print(f'Objective defined to be maximizing {objective_reaction.id}: {objective_reaction.reaction}')

# Investigating the base line max production and growth rate
#with model:
#    base_max_growth = model.slim_optimize()
#    print(base_max_growth)
#    base_max_production = model.slim_optimize()
#    print(base_max_production)

In [13]:
#model.solver = "glpk" #specifying the solver for optimizing to be used as GLPK (GNU Linear Programming Kit)
#biomass = model.reactions.BIOMASS_BS_10 #specifying our objective: optimizing biomass formation
#biomass.lower_bound = 0.1 
#target = model.metabolites.get_by_id("dihydroartaci_c") #final product
#demand = model.reactions.get_by_id("DM_dihydroartaci_c") #this might not yield anything...
#model.objective = demand
#from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
#prod_yield = product_yield(model.reactions.ALDH1_CYP71AV1, model.reactions.r_1714) #OBS I DON'T KNOW WHAT THIS LAST REACTION SHOULD BE (r_1714??)
#print(prod_yield)

#By default this method looks to replace NADH and NADPH. You can set different co-factor pairs via the parameter cofactor_id_swaps=([from], [to]) which requires the metabolite identifiers in the model of the pairs that you want to swap. Please look at the documentation for more info. #CHANGE THIS TEXT, IT IS COPY-PASTE

In [14]:
# Swapping NADPH to NADH                                                                               
#swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True, cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']))
#%%time
#result = swap_opt.run(max_size=5) #can be changed, but I'M NOT QUITE SURE WHAT EXACTLY CHANGES (looks like time in min)

In [15]:
#print(f'The results of co-factor swapping are: '{result})

We can now try to remake all reactions in the implemented pathway using NADPH, changing them to instead require NADH. 
This means we must change reactions 3,4,5,6...

#We can now try to remake the r_0659 reaction to increase the fluxes.

Starting over....

